INSTALAÇÃO DE DEPENDÊNCIAS E BIBLIOTECAS QUE SERÃO NECESSÁRIAS

In [30]:
!pip install pydantic
!pip install pydantic[email]

IMPORTAÇÕES DAS BIBLIOTECAS QUE SERÃO UTILIZADAS NO CÓDIGO

In [31]:
import enum
import hashlib
import re
from typing import Any
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

CRIAÇÃO DE EXPRESSÕES REGULARES PARA VALIDAR NOMES E SENHAS

In [ ]:
# Define uma regra para as senhas, a senha deve possuir no mínimo 8 caracteres,
# sendo pelo menos 1 letra maiúscula, 1 letra minúscula e 1 número.
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[A-Z])(?=.*[a-z])(?=.*\d).{8,}$")
# Define uma regra para os nomes, o nome deve possuir no mínimo 2 caracteres.
VALID_NAME_REGEX = re.compile(r"^[A-Za-z]{2,}$")

CRIAÇÃO DE UMA CLASSE DE CLASSIFICAÇÃO DE PERMISSÕES

In [ ]:
# Essa classe serve para designar uma função para o Usuário
# O ordem de "importância" segue a ordem de declaração
# Ou seja, Author < Editor < Admin < SuperAdmin
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

CRIAÇÃO DA CLASSE USUÁRIO COM VALIDAÇÕES PERSONALIZADAS PARA OS DADOS

In [ ]:
class User(BaseModel):
    # O uso do 'str' é para garantir que o campo seja uma string, além disso ele oferce um exemplo de um valor válido para essa variável
    name: str = Field(examples=["Arjan"])
    # O uso do 'EmailStr' é para garantir que o campo seja um email
    # O 'Field' é utilizado para adicionar metadados e resrições ao campo, servindo como uma indicação do formato esperado para esse campo
    email: EmailStr = Field(
        # Ele oferce um exemplo de um valor válido para essa variável
        examples=["user@arjancodes.com"],
        # Aqui é feita uma descrição do que é esperado para esse campo
        description="The email address of the user",
        #O parâmetro Frozen é utilizado para garantir que o objeto não seja alterado após a sua criação
        frozen=True,
    )
    #O uso do 'SecretStr' é para garantir que o campo seja uma senha, ou seja, caso ela seja exibida em algum lugar, ela será exibida como '********'
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    #O atributo role é utilizado para designar a função do usuário
    role: Role = Field(default=None, description="The role of the user", examples=[1,2,4,8])
    
    #Esses 'fields_validator' são validações personalizadas feitas para garantir que os dados inseridos sejam válidos
    @field_validator("name")
    @classmethod
    #Como é um método de classe, ele recebe a classe como primeiro argumento e o valor a ser validado como segundo argumento
    #O retorno desse método é uma string
    def validate_name(cls, v: str) -> str:
        # Aqui é feita uma verificação, a partir de correspondência com a "regra" da REGEX imposta, 
        # para garantir que o nome inserido seja válido
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letter and be at least 2 characters long"
            )
        return v


    #Aqui é feita uma validação para garantir que o valor inserido para o usuário seja válido
    #Como o mode dessa validação é 'before', ela é feita antes da validação padrão do Pydantic
    @field_validator("role", mode="before")
    @classmethod
    #Como é um método de classe, ele recebe a classe como primeiro argumento 
    #e o segundo parâmetro é o valor a ser validado, que pode ser um inteiro, uma string ou um Role
    #O retorno desse método é um Role, ou seja, um valor de função 
    def validate_role(cls, v: int | str | Role) -> Role:
        #Esse dicionário é utilizado para mapear os valores possíveis para a função
        #Se v for um inteiro, ele é convertido para um Role, se for uma string, 
        #ele é convertido para um Role - por meio do nome da role -, se for um Role, ele é retornado
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            #Aqui é feita a conversão do valor inserido para um Role
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {",".join([x.name for x in Role])}"
                )
        
    #Aqui é feita uma validação para garantir que o valor inserido para o usuário seja válido
    #Como o mode dessa validação é 'before', ela é feita antes da validação padrão do Pydantic
    @model_validator(mode="before")
    @classmethod
     #Como é um método de classe, ele recebe a classe como primeiro argumento 
     #e o valor a ser validado como segundo argumento, em forma de dicionário  
     #contendo  os dados do usuário- onde 
     #as chaves são strings e os valores podem ser de qualquer tipo.
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        #Aqui é feita uma verificação para garantir que o nome e a senha foram inseridos
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        #Aqui é feita uma verificação para garantir que a senha não contenha o nome do usuário
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        #Aqui é feita uma verificação para garantir que a senha inserida seja
        #válida, ou seja, contenha 8 caracteres, 1 maiúscula, 1 minúscula e 1 número
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        #Aqui é feita uma criptografia da senha inserida, para melhorar a segurança
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

CRIAÇÃO DE UMA FUNÇÃO DE VALIDAÇÃO

In [35]:
# A criação da função 'model_validate' é criada com um parametro 'data' que é do tipo 'dict' e não retorna nada
def validate(data: dict[str, Any]) -> None:
    try:
        # Aqui é feita a validação do modelo 'User', se os dados estão de acordo com a estrutura do User
        user = User.model_validate(data)
        # Caso os dados estejam corretos, o usuário é exibido
        print(user)
    except ValidationError as e:
        print('User is invalid')
        # Caso os dados estejam incorretos, o erro será exibido e é mostrado que o usuário é inválido
        print(e)

CRIAÇÃO DA FUNÇÃO MAIN (PRINCIPAL)

In [36]:
def main() -> None:
    """
    Essa função é responsável por chamar a função 'validate' com dados corretos e incorretos
    Ela possui dados de acordo com o que foi pedido e
    dados que não estão de acordo com o que foi pedido
    Exemplo:
     good_data = {
        "name": "Arjan",
        "email": "example@arjancodes.com",
        "password": "Password123",
    }
     validate(good_data)


    main() -> name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
    
    """
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    # Aqui é chamada a função 'validate' com os dados corretos e incorretos
    # Para cada exemplo, é exibido o nome do exemplo e o resultado da validação
    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

EXECUÇÃO DA FUNÇÃO MAIN 

In [37]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author,Editor,Admin,SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

bad_data
User is invalid
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error

bad_name
User is invalid
1 validation error for User
name
  Value error, Name is invalid, must contain only letter and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://errors.py